# Multivariate ARIMA Prediction - single asset
## 1. Load libraries

In [10]:
#import libraries
import pandas as pd
from pandas import DataFrame 
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 18,10
%matplotlib inline

#import itertools
from statsmodels.tsa.arima_model import ARIMA
#from statsmodels.tsa.stattools import acf, pacf, adfuller
#from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.var_model import VAR


import numpy as np
from numpy import log
rcParams['figure.figsize'] = 18,10



## 2. Load and visualize time series dataset
### 2.1 Load data
Asset data of weekly frequency named xxx.csv is downloaded from Yahoo Finance UK is loaded into the notebook by the def GetData function. The loaded data is presented as a float32 datatype in a DataFrame format.

In [2]:
%store -r data_IVV
%store -r data_SHY
%store -r data_VNQ
%store -r data_GLD
%store -r data_VIX
%store -r vol_IVV
%store -r vol_SHY
%store -r vol_VNQ 
%store -r vol_GLD 


In [3]:
%store -r data_savings


In [4]:
data_m = pd.concat([data_savings,data_VIX],axis=1)
data_m = data_m.fillna(method='ffill')

In [5]:
GLD_all = pd.concat([data_GLD,vol_GLD,data_m],axis=1).astype('float32').dropna()


In [6]:
GLD_all

,Adj Close,Volume,Value,Adj Close
Date,,,,
2009-06-01,93.709999,82888896.0,6.7,30.040001
2009-06-08,92.169998,64057600.0,6.7,29.770000
2009-06-15,91.900002,63132500.0,6.7,30.809999
2009-06-22,92.290001,58891900.0,6.7,31.170000
2009-06-29,91.250000,48826600.0,6.7,25.350000
2009-07-06,89.580002,54662800.0,6.0,29.000000
2009-07-13,91.930000,37747200.0,6.0,26.309999
2009-07-20,93.410004,47255100.0,6.0,24.400000
2009-07-27,93.349998,46357200.0,6.0,24.280001


#### Stationarity Checks
Johansen Test for Cointegration (Alternative to Dickey Fuller Test)
[more on the math](https://blog.quantinsti.com/johansen-test-cointegration-building-stationary-portfolio/)


In [8]:
coint_johansen(GLD_all,-1,1).eig

array([1.89612927e-01, 6.26506458e-02, 2.90486974e-02, 1.68749168e-05])

Train test split

In [13]:
train = GLD_all[:int(0.6*(len(GLD_all)))]
test = GLD_all[int(0.6*(len(GLD_all))):]


Fitting the model

In [14]:
model = VAR(endog=train)
model_fit = model.fit()

//anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Make prediction on test set

In [15]:
pred = model_fit.forecast(model_fit.y, steps=len(test))


Convert predictions to DataFrame

In [21]:
prediction = pd.DataFrame(pred)


Check root mean sq error

In [22]:
col = prediction.columns()
for i in cols:
    print('rmse value for', i, 'is : ', sqrt(mean_squared_error(pred[i], valid[i])))

TypeError: 'RangeIndex' object is not callable

Final predictions on price

In [25]:
model = VAR(endog=GLD_all)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[1.21337045e+02 2.82905254e+07 7.95062887e+00 1.59510268e+01]]


//anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
